In [1]:
import pandas as pd
import numpy as np
import string
import re

from datasets import load_dataset, ClassLabel

from nlp_id.stopword import StopWord
from nlp_id.tokenizer import Tokenizer
from nlp_id.lemmatizer import Lemmatizer

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import bahasa baku dan Sarcastic comments

In [2]:
formal_words_id = load_dataset('theonlydo/indonesia-slang')
print(formal_words_id['train'])


# sarcasm_id = load_dataset('w11wo/twitter_indonesia_sarcastic')
# print(sarcasm_id)

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['slang', 'formal'],
    num_rows: 4412
})


In [3]:
corpus = pd.read_csv('../datasets/all_comments_temp.csv', encoding='utf-8')

corpus.head()


,id,username,comment,num_of_likes,published_at
0,N3JZxBTC1Zw,@alvinvinoza,Naikin gajinya 🔥🔥,962,2025-03-12T14:42:39Z
1,9YFbtgNsSeA,@MuklasinMuklasin-k6w,"Kalo tidak ada hukuman mati, sampek kiamat pun...",847,2025-02-25T22:31:15Z
2,9YFbtgNsSeA,@infotodax,"Gua gak kebayang, Gaji elu berapa... Tapi kese...",683,2025-02-26T03:59:00Z
3,N3JZxBTC1Zw,@febbriyansaputra4132,Tolong host nya di support dan di lindungi ini,552,2025-03-19T13:20:14Z
4,N3JZxBTC1Zw,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...,535,2025-03-06T16:28:31Z


In [4]:
print(corpus.info())
print('\nMissing Values: \n',corpus.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7554 entries, 0 to 7553
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            7554 non-null   object
 1   username      7554 non-null   object
 2   comment       7554 non-null   object
 3   num_of_likes  7554 non-null   int64 
 4   published_at  7554 non-null   object
dtypes: int64(1), object(4)
memory usage: 295.2+ KB
None

Missing Values: 
 id              0
username        0
comment         0
num_of_likes    0
published_at    0
dtype: int64


In [5]:
df = corpus[['username', 'comment']]
df

,username,comment
0,@alvinvinoza,Naikin gajinya 🔥🔥
1,@MuklasinMuklasin-k6w,"Kalo tidak ada hukuman mati, sampek kiamat pun..."
2,@infotodax,"Gua gak kebayang, Gaji elu berapa... Tapi kese..."
3,@febbriyansaputra4132,Tolong host nya di support dan di lindungi ini
4,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...
...,...,...
7549,hedianti,@KPK_RI Halah..selama korupsi ga hukum mati......
7550,Charles39953441,@KPK_RI beberapa Dirut pertamina jadi tersangk...
7551,Aryabarcelona,@KPK_RI 10 tahun baru terungkap beneran gak si...
7552,harley_doni,@KPK_RI Masak KA sebodoh itu ya. Apakah mmg me...


### convert corpus to lower case

In [6]:
df['comment'] = df['comment'].apply(lambda content: " ".join(words.lower() for words in content.split()))
df['comment']

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\2338691186.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(lambda content: " ".join(words.lower() for words in content.split()))


0                                       naikin gajinya 🔥🔥
1       kalo tidak ada hukuman mati, sampek kiamat pun...
2       gua gak kebayang, gaji elu berapa... tapi kese...
3          tolong host nya di support dan di lindungi ini
4       harusnya berita ya kayak begini dibawa santai ...
                              ...                        
7549    @kpk_ri halah..selama korupsi ga hukum mati......
7550    @kpk_ri beberapa dirut pertamina jadi tersangk...
7551    @kpk_ri 10 tahun baru terungkap beneran gak si...
7552    @kpk_ri masak ka sebodoh itu ya. apakah mmg me...
7553    @kpk_ri nah kalo semua bisa di tindak harus je...
Name: comment, Length: 7554, dtype: object

## Cleaning mark quotion etc.

In [7]:
tokenizer = Tokenizer()
stopword = StopWord()
lemmatizer = Lemmatizer()

In [8]:
import re
import string

def clean_mark(data):
    # Menghapus mention (@username)
    data = re.sub(r'@[A-Za-z0-9]+', '', data)
    
    # Menghapus hashtag (#hashtag)
    data = re.sub(r'#[A-Za-z0-9]+', '', data)
    
    # Menghapus URL
    data = re.sub(r'http\S+', '', data)
    
    # Menghapus angka
    data = re.sub(r'[0-9]+', '', data)

    # Menghapus RT (retweet)
    data = re.sub(r'RT\s+', '', data)
    
    # Menghapus karakter non-huruf dan non-angka
    data = re.sub(r'[^A-Za-z0-9]', ' ', data)

    # Menghapus karakter yang berulang lebih dari dua kali
    data = re.sub(r'(.)\1+', r'\1\1', data)
    
    # Menghapus duplikasi kata berturut-turut
    data = re.sub(r'\b(\w+)( \1\b)+', r'\1', data)
    
    # Menghapus tanda baca berturut-turut
    data = re.sub(r'[\?\.\!]+(?=[\?.\!])', '', data)
    
    # Menghapus tanda baca
    data = data.translate(str.maketrans('', '', string.punctuation))
    
    # Menghapus spasi di awal dan akhir
    data = data.strip()
    
    return data


def remove_emojis(text):
    emoji_pattern = re.compile(
        u'['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbol
        u'\U0001F680-\U0001F6FF'  # transports and map
        u'\U0001F700-\U0001F77F'  # alchemist symbols
        u'\U0001F780-\U0001F7FF'  # geometry symbols
        u'\U0001F800-\U0001F8FF'  # additional symbols
        u'\U0001F900-\U0001F9FF'  # faces, hands, etc.
        u'\U0001FA00-\U0001FA6F'  # additional faces
        u'\U0001FA70-\U0001FAFF'  # additional symbols
        u'\U00002702-\U000027B0'  # hand gestures, symbols
        u'\U000024C2-\U0001F251'  # general emojis
        u'\U0001F004-\U0001F0CF'  # playing card symbols
        u'\U0001F000-\U0001F02F'  # Mahjong
        u'\U0001F1E0-\U0001F1FF'  # Flags
        u'\U0001F200-\U0001F251'  # Enclosed characters
        u']', re.UNICODE)
    
    return re.sub(emoji_pattern, '', text)


In [9]:
# df[['comment','username']] = df[['comment','username']].apply(lambda cols: cols.apply(clean_mark))
df['comment'] = df['comment'].apply(clean_mark)
df.head()

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\4133955545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(clean_mark)


,username,comment
0,@alvinvinoza,naikin gajinya
1,@MuklasinMuklasin-k6w,kalo tidak ada hukuman mati sampek kiamat pun...
2,@infotodax,gua gak kebayang gaji elu berapa tapi kesera...
3,@febbriyansaputra4132,tolong host nya di support dan di lindungi ini
4,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...


In [10]:
df['comment'] = df['comment'].apply(remove_emojis)
df.head()

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\3789465410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(remove_emojis)


,username,comment
0,@alvinvinoza,naikin gajinya
1,@MuklasinMuklasin-k6w,kalo tidak ada hukuman mati sampek kiamat pun...
2,@infotodax,gua gak kebayang gaji elu berapa tapi kesera...
3,@febbriyansaputra4132,tolong host nya di support dan di lindungi ini
4,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...


### Tokenize text

In [11]:
df['comment'] = df['comment'].apply(lambda txt:tokenizer.tokenize(txt))

df

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\4112932001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(lambda txt:tokenizer.tokenize(txt))


,username,comment
0,@alvinvinoza,"[naikin, gaji, nya]"
1,@MuklasinMuklasin-k6w,"[kalo, tidak, ada, hukuman, mati, sampek, kiam..."
2,@infotodax,"[gua, gak, kebayang, gaji, elu, berapa, tapi, ..."
3,@febbriyansaputra4132,"[tolong, host, nya, di, support, dan, di, lind..."
4,@chalvit4162,"[harusnya, berita, ya, kayak, begini, dibawa, ..."
...,...,...
7549,hedianti,"[ri, halah, selama, korupsi, ga, hukum, mati, ..."
7550,Charles39953441,"[ri, beberapa, dirut, pertamina, jadi, tersang..."
7551,Aryabarcelona,"[ri, tahun, baru, terungkap, beneran, gak, si,..."
7552,harley_doni,"[ri, masak, ka, sebodoh, itu, ya, apakah, mmg,..."


### Replace Slang to Formal words

In [12]:
df['comment']

0                                     [naikin, gaji, nya]
1       [kalo, tidak, ada, hukuman, mati, sampek, kiam...
2       [gua, gak, kebayang, gaji, elu, berapa, tapi, ...
3       [tolong, host, nya, di, support, dan, di, lind...
4       [harusnya, berita, ya, kayak, begini, dibawa, ...
                              ...                        
7549    [ri, halah, selama, korupsi, ga, hukum, mati, ...
7550    [ri, beberapa, dirut, pertamina, jadi, tersang...
7551    [ri, tahun, baru, terungkap, beneran, gak, si,...
7552    [ri, masak, ka, sebodoh, itu, ya, apakah, mmg,...
7553    [ri, nah, kalo, semua, bisa, di, tindak, harus...
Name: comment, Length: 7554, dtype: object

In [13]:
slang2formal = dict(zip(formal_words_id['train']['slang'], formal_words_id['train']['formal']))
print(list(slang2formal.items())[:10])

def replace_slang(words, slang_to_formal):
    return [slang_to_formal.get(word.lower(), word) for word in words]

df['comment'] = df['comment'].apply(lambda txt: replace_slang(txt, slang2formal))

print(df)

[('woww', 'wow'), ('aminn', 'amin'), ('met', 'selamat'), ('netaas', 'menetas'), ('keberpa', 'keberapa'), ('eeeehhhh', 'eh'), ('kata2nyaaa', 'kata-katanya'), ('hallo', 'halo'), ('kaka', 'kakak'), ('ka', 'kak')]
                   username                                            comment
0              @alvinvinoza                                [naikin, gaji, nya]
1     @MuklasinMuklasin-k6w  [kalo, tidak, ada, hukuman, mati, sampai, kiam...
2                @infotodax  [gua, enggak, terbayang, gaji, lu, berapa, tap...
3     @febbriyansaputra4132  [tolong, host, nya, di, support, dan, di, lind...
4              @chalvit4162  [harusnya, berita, ya, kayak, begini, dibawa, ...
...                     ...                                                ...
7549               hedianti  [ri, halah, selama, korupsi, enggak, hukum, ma...
7550        Charles39953441  [ri, beberapa, dirut, pertamina, jadi, tersang...
7551          Aryabarcelona  [ri, tahun, baru, terungkap, benaran, enggak, ...


C:\Users\alber\AppData\Local\Temp\ipykernel_15104\2188415782.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(lambda txt: replace_slang(txt, slang2formal))


In [14]:
# def replace_slang(txt, slang_to_formal):
#     words = txt.split()
    
#     words = [slang_to_formal.get(word.lower(), word) for word in words]


#     return ' '.join(words)


# examp = 'met pagi kasi kata2nya'
# upd = replace_slang(examp, slang2formal)
# print(upd)

### Remove StopWords

In [15]:
def remove_stopwords(txt):
    txt = [word for word in txt if word.lower() not in stopword.get_stopword()]

    return txt

df['comment'] = df['comment'].apply(remove_stopwords)
df

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\62721408.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(remove_stopwords)


,username,comment
0,@alvinvinoza,"[naikin, gaji]"
1,@MuklasinMuklasin-k6w,"[kalo, hukuman, mati, kiamat, negeri, korupsi]"
2,@infotodax,"[enggak, terbayang, gaji, lu, keserakahan, lu]"
3,@febbriyansaputra4132,"[tolong, host, support, lindungi]"
4,@chalvit4162,"[berita, kayak, dibawa, santai, tajam, abang, ku]"
...,...,...
7549,hedianti,"[ri, korupsi, enggak, hukum, mati, cuman, remi..."
7550,Charles39953441,"[ri, dirut, pertamina, tersangka, menjabat, lqgi]"
7551,Aryabarcelona,"[ri, terungkap, benaran, enggak, kerja]"
7552,harley_doni,"[ri, masak, kak, sebodoh, membodohkan, janji, ..."


### lemmatize words

In [16]:
def lemmatize_words(txt):
    return [lemmatizer.lemmatize(word) for word in txt]


df['comment'] = df['comment'].apply(lemmatize_words)
df

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\1827849294.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(lemmatize_words)


,username,comment
0,@alvinvinoza,"[naikin, gaji]"
1,@MuklasinMuklasin-k6w,"[kalo, hukum, mati, kiamat, negeri, korupsi]"
2,@infotodax,"[enggak, bayang, gaji, lu, serakah, lu]"
3,@febbriyansaputra4132,"[tolong, host, support, lindung]"
4,@chalvit4162,"[berita, kayak, bawa, santai, tajam, abang, ku]"
...,...,...
7549,hedianti,"[ri, korupsi, enggak, hukum, mati, cuman, remi..."
7550,Charles39953441,"[ri, dirut, pertamina, sangka, jabat, lqgi]"
7551,Aryabarcelona,"[ri, ungkap, benar, enggak, kerja]"
7552,harley_doni,"[ri, masak, kak, bodoh, bodoh, janji, komisi, ..."


### Stemming and Joining(detokenize) after cleansing

In [17]:
def stemming_words(txt):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    res = [stemmer.stem(word) for word in txt]
    clean_txt = " ".join(res)

    return clean_txt

df['comment_cleansing'] = df['comment'].apply(lambda x: stemming_words(x))
df

C:\Users\alber\AppData\Local\Temp\ipykernel_15104\1708580434.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment_cleansing'] = df['comment'].apply(lambda x: stemming_words(x))


,username,comment,comment_cleansing
0,@alvinvinoza,"[naikin, gaji]",naikin gaji
1,@MuklasinMuklasin-k6w,"[kalo, hukum, mati, kiamat, negeri, korupsi]",kalo hukum mati kiamat negeri korupsi
2,@infotodax,"[enggak, bayang, gaji, lu, serakah, lu]",enggak bayang gaji lu serakah lu
3,@febbriyansaputra4132,"[tolong, host, support, lindung]",tolong host support lindung
4,@chalvit4162,"[berita, kayak, bawa, santai, tajam, abang, ku]",berita kayak bawa santai tajam abang ku
...,...,...,...
7549,hedianti,"[ri, korupsi, enggak, hukum, mati, cuman, remi...",ri korupsi enggak hukum mati cuman remisi engg...
7550,Charles39953441,"[ri, dirut, pertamina, sangka, jabat, lqgi]",ri dirut pertamina sangka jabat lqgi
7551,Aryabarcelona,"[ri, ungkap, benar, enggak, kerja]",ri ungkap benar enggak kerja
7552,harley_doni,"[ri, masak, kak, bodoh, bodoh, janji, komisi, ...",ri masak kak bodoh bodoh janji komisi mwnggiur...


In [18]:
df.to_csv('../datasets/cleaned_comment_public.csv', index=False)

https://www.kaggle.com/code/dewanakretarta/transformer-classification-gojek

https://www.kaggle.com/code/alkidiarete/fastai-gojek-reviews

https://www.kaggle.com/code/rizkia14/analisis-sentimen-unsupervised-lexical#Textblob-Lexicon

https://www.kaggle.com/code/miftahurrizki/eda-topic-modelling-using-pyldavis/notebook

https://www.kaggle.com/code/dewanakretarta/transformer-classification-mobile-legends